In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [25]:
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn import metrics
from sklearn.cross_validation import cross_val_score

/home/andraa/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
from datetime import datetime

In [3]:
pd.options.mode.chained_assignment = None

In [4]:
################# members csv #######################

In [5]:
members_df = pd.read_csv("/media/andraa/10160545101605451/kaggle/WSDM-kaggle/data/members.csv")

In [6]:
members_df.head()

,msno,city,bd,gender,registered_via,registration_init_time,expiration_date
0,URiXrfYPzHAlk+7+n7BOMl9G+T7g8JmrSnT/BU8GmEo=,1,0,NaN,9,20150525,20150526
1,U1q0qCqK/lDMTD2kN8G9OXMtfuvLCey20OAIPOvXXGQ=,1,0,NaN,4,20161221,20161224
2,W6M2H2kAoN9ahfDYKo3J6tmsJRAeuFc9wl1cau5VL1Q=,1,0,NaN,4,20160306,20160309
3,1qE5+cN7CUyC+KFH6gBZzMWmM1QpIVW6A43BEm98I/w=,5,17,female,4,20161031,20161107
4,SeAnaZPI+tFdAt+r3lZt/B8PgTp7bcG/1os39u4pLxs=,1,0,NaN,4,20170202,20170205


In [7]:
members_df["gender"].fillna("no_gen", inplace = True)

In [8]:
members_df.dtypes

msno                      object
city                       int64
bd                         int64
gender                    object
registered_via             int64
registration_init_time     int64
expiration_date            int64
dtype: object

In [9]:
# see how many have set birth to be 0
len(members_df[members_df["bd"] == 0])

3421570

In [10]:
# see how many have declared an abberant age
len(members_df[(members_df["bd"] <=8 ) | (members_df["bd"] >=99 ) ])

3426211

In [11]:
# see how many have not declared age
len(members_df[members_df["gender"] == "no_gen"])

3354778

In [12]:
# create an indicator for those who have declared wrongly their age
members_df["false_age"] = 0


In [13]:
members_df[(members_df["bd"] <=8 ) | (members_df["bd"] >=99 )]["bd"] = 1

In [14]:
# convert the date of the registration_init and the expiration_date from int to datetime
def convert_date(x):
    x = str(x)
    date = datetime(year=int(x[0:4]), month=int(x[4:6]), day=int(x[6:8]))
    #date = datetime.strptime(date, "%Y-%m-%d")
    return date

members_df["registration_init_time"] = members_df["registration_init_time"].apply(lambda x : convert_date(x))

In [15]:
members_df["expiration_date"] = members_df["expiration_date"].apply(lambda x : convert_date(x))

In [16]:
members_df.head()

,msno,city,bd,gender,registered_via,registration_init_time,expiration_date,false_age
0,URiXrfYPzHAlk+7+n7BOMl9G+T7g8JmrSnT/BU8GmEo=,1,0,no_gen,9,2015-05-25,2015-05-26,0
1,U1q0qCqK/lDMTD2kN8G9OXMtfuvLCey20OAIPOvXXGQ=,1,0,no_gen,4,2016-12-21,2016-12-24,0
2,W6M2H2kAoN9ahfDYKo3J6tmsJRAeuFc9wl1cau5VL1Q=,1,0,no_gen,4,2016-03-06,2016-03-09,0
3,1qE5+cN7CUyC+KFH6gBZzMWmM1QpIVW6A43BEm98I/w=,5,17,female,4,2016-10-31,2016-11-07,0
4,SeAnaZPI+tFdAt+r3lZt/B8PgTp7bcG/1os39u4pLxs=,1,0,no_gen,4,2017-02-02,2017-02-05,0


In [17]:
# define the difference between the registration and the expiration
members_df["longivity"] = (members_df["expiration_date"]  - members_df["registration_init_time"] ).astype('timedelta64[D]')

In [18]:
members_df.head()

,msno,city,bd,gender,registered_via,registration_init_time,expiration_date,false_age,longivity
0,URiXrfYPzHAlk+7+n7BOMl9G+T7g8JmrSnT/BU8GmEo=,1,0,no_gen,9,2015-05-25,2015-05-26,0,1.0
1,U1q0qCqK/lDMTD2kN8G9OXMtfuvLCey20OAIPOvXXGQ=,1,0,no_gen,4,2016-12-21,2016-12-24,0,3.0
2,W6M2H2kAoN9ahfDYKo3J6tmsJRAeuFc9wl1cau5VL1Q=,1,0,no_gen,4,2016-03-06,2016-03-09,0,3.0
3,1qE5+cN7CUyC+KFH6gBZzMWmM1QpIVW6A43BEm98I/w=,5,17,female,4,2016-10-31,2016-11-07,0,7.0
4,SeAnaZPI+tFdAt+r3lZt/B8PgTp7bcG/1os39u4pLxs=,1,0,no_gen,4,2017-02-02,2017-02-05,0,3.0


In [ ]:
################# transactions csv #######################

In [4]:
transactions_df = pd.read_csv("/media/andraa/10160545101605451/kaggle/WSDM-kaggle/data/transactions.csv")

In [5]:
transactions_df.head()

,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel
0,YyO+tlZtAXYXoZhNr3Vg3+dfVQvrBVGO8j1mfqe4ZHc=,41,30,129,129,1,20150930,20151101,0
1,AZtu6Wl0gPojrEQYB8Q3vBSmE2wnZ3hi1FbK1rQQ0A4=,41,30,149,149,1,20150930,20151031,0
2,UkDFI97Qb6+s2LWcijVVv4rMAsORbVDT2wNXF0aVbns=,41,30,129,129,1,20150930,20160427,0
3,M1C56ijxozNaGD0t2h68PnH2xtx5iO5iR2MVYQB6nBI=,39,30,149,149,1,20150930,20151128,0
4,yvj6zyBUaqdbUQSrKsrZ+xNDVM62knauSZJzakS9OW4=,39,30,149,149,1,20150930,20151121,0


In [ ]:
# transform to date mem

In [38]:
transactions_df["membership_expire_date"] = transactions_df["membership_expire_date"].apply(lambda x : convert_date(x))
transactions_df["transaction_date"] = transactions_df["transaction_date"].apply(lambda x : convert_date(x))
transactions_df["early_action"] =(transactions_df["membership_expire_date"] - transactions_df["transaction_date"]).astype('timedelta64[D]')

In [40]:
transactions_df[transactions_df["early_action"]>0]

,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel,discount,early_action
0,YyO+tlZtAXYXoZhNr3Vg3+dfVQvrBVGO8j1mfqe4ZHc=,41,30,129,129,1,2015-09-30,2015-11-01,0,0,32.0
1,AZtu6Wl0gPojrEQYB8Q3vBSmE2wnZ3hi1FbK1rQQ0A4=,41,30,149,149,1,2015-09-30,2015-10-31,0,0,31.0
2,UkDFI97Qb6+s2LWcijVVv4rMAsORbVDT2wNXF0aVbns=,41,30,129,129,1,2015-09-30,2016-04-27,0,0,210.0
3,M1C56ijxozNaGD0t2h68PnH2xtx5iO5iR2MVYQB6nBI=,39,30,149,149,1,2015-09-30,2015-11-28,0,0,59.0
4,yvj6zyBUaqdbUQSrKsrZ+xNDVM62knauSZJzakS9OW4=,39,30,149,149,1,2015-09-30,2015-11-21,0,0,52.0
5,KN7I82kjY0Tn76Ny95ncqBUVbO7b8AXrOPqZutKpxIM=,21,30,149,149,1,2015-09-30,2015-11-07,0,0,38.0
6,m5ptKif9BjdUghHXXomSezy5ohJiHm85PE13f/3kQaw=,39,30,149,149,1,2015-09-30,2015-11-28,0,0,59.0
7,uQxbyACsPOEkTIrv9jZgoGXelGBW81ZsSZKy9fhj5Z8=,39,30,149,149,1,2015-09-30,2015-11-25,0,0,56.0
8,LUPRfoE2r3WwVWhYO/TqQhjrL/qP6CO+/ORUlr7yNc0=,39,30,149,149,1,2015-09-30,2015-12-22,0,0,83.0
9,pMVjPLgVknaJYm9L0xUwxA2jmhpK7gfhGnsmmwX1mrE=,39,30,149,149,1,2015-09-30,2015-11-18,0,0,49.0


In [6]:
# we want to see what is the discount distribution
transactions_df["discount"] = transactions_df["plan_list_price"] - transactions_df["actual_amount_paid"]

In [7]:
transactions_df.describe()

,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel,discount
count,2.154775e+07,2.154775e+07,2.154775e+07,2.154775e+07,2.154775e+07,2.154775e+07,2.154775e+07,2.154775e+07,2.154775e+07
mean,3.893310e+01,3.133906e+01,1.398850e+02,1.419873e+02,8.519661e-01,2.015760e+07,2.015862e+07,3.976523e-02,-2.102305e+00
std,3.507936e+00,3.035649e+01,1.309647e+02,1.324824e+02,3.551336e-01,6.155979e+03,7.755753e+03,1.954072e-01,4.753888e+01
min,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.015010e+07,1.970010e+07,0.000000e+00,-2.000000e+03
25%,3.800000e+01,3.000000e+01,9.900000e+01,9.900000e+01,1.000000e+00,2.015091e+07,2.015102e+07,0.000000e+00,0.000000e+00
50%,4.100000e+01,3.000000e+01,1.490000e+02,1.490000e+02,1.000000e+00,2.016032e+07,2.016043e+07,0.000000e+00,0.000000e+00
75%,4.100000e+01,3.000000e+01,1.490000e+02,1.490000e+02,1.000000e+00,2.016092e+07,2.016103e+07,0.000000e+00,0.000000e+00
max,4.100000e+01,4.500000e+02,2.000000e+03,2.000000e+03,1.000000e+00,2.017023e+07,2.017033e+07,1.000000e+00,6.990000e+02


In [37]:
transactions_df[transactions_df["discount"] < -150]

,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel,discount
4229,mvqeUO8IspCrNTVlZ4fRwBTS1YVBYbsGhEDmWgPF3cg=,32,0,0,536,0,20150501,20151031,0,-536
4467,56+symU54Q/1LYyd9XD5z0SrcR4l4lKSSaqZa8Nov4A=,38,0,0,894,0,20150428,20151109,0,-894
5283,71upN4HTnpKSZGZJCBPj7Kj8hgJKsWZUQRqop7axDTI=,22,0,0,1788,0,20150505,20160621,0,-1788
9760,/TgmB07TiHVZSo7KwPhmAcQMIHeHt/T9VYws8jfx1do=,13,0,0,1788,0,20150510,20160623,0,-1788
10713,BDNzoO0NTjzLiySPA7BqzuqfUU2H/0gvLEv8dHrcMVo=,32,0,0,799,0,20150426,20151028,0,-799
10714,/Atus9wCMz7RJjmsAUm1XKKodFP+4oJIh7s4y7KEpBY=,22,0,0,894,0,20150426,20151114,0,-894
13559,AoQ1+fHf6yU/dp2acaOEM5YjuSeuFNm5irjJ//FFCXs=,22,0,0,1788,0,20150501,20160617,0,-1788
14332,eT1YUatJIHuWFCm+8Nr/BX/aLSR+hRj4N9P93Hnm/a8=,38,0,0,894,0,20150504,20151116,0,-894
20633,DQJkojQ2W4xlxndsM18uc3dZMUqecDwEdw7rzR0zsiE=,32,0,0,536,0,20150428,20151028,0,-536
23404,/0jLhuEIn2g3nTcFoYuqO0Njtpm04mY3gW5j+Ua5HG4=,32,0,0,894,0,20150501,20151115,0,-894


In [8]:
transactions_df.groupby("msno").last().reset_index()

,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel,discount
0,+++FOrTS7ab3tIgIh8eWwX4FqRv8w/FoiOuyXsFvphY=,35,7,0,0,0,20160909,20160914,0,0
1,+++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=,38,410,1788,1788,0,20151121,20170104,0,0
2,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,41,30,99,99,1,20161215,20170115,0,0
3,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,39,30,149,149,1,20150630,20150819,0,0
4,+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,41,30,149,149,1,20160126,20160226,0,0
5,++/9R3sX37CjxbY/AaGvbwr3QkwElKBCtSvVzhCBDOk=,41,30,149,149,1,20160815,20160915,0,0
6,++/Gw1B9K+XOlB3hLTloeUK2QlCa2m+BJ8TrzGf7djI=,32,195,894,894,0,20150819,20160301,0,0
7,++/TR7WI15q2ZCtOXmoap7jR+kEhbMVE5swOqsfqpqI=,11,31,149,149,1,20150124,20150225,0,0
8,++/UDNo9DLrxT8QVGiDi1OnWfczAdEwThaVyD0fXO50=,39,30,149,149,1,20151102,20151223,0,0
9,++/ZHqwUNa7U21Qz+zqteiXlZapxey86l6eEorrak/g=,14,30,149,149,1,20160430,20160604,0,0


In [9]:
train_df = pd.read_csv("/media/andraa/10160545101605451/kaggle/WSDM-kaggle/data/train.csv")

In [12]:
columns_to_merge = ["msno","payment_method_id", "payment_plan_days", "plan_list_price", "is_auto_renew"]
train_transactions = train_df.merge(transactions_df.groupby("msno").last().reset_index()[columns_to_merge])

In [13]:
train_transactions.to_csv("/media/andraa/10160545101605451/kaggle/WSDM-kaggle/data/train_transactions.csv")

In [19]:
train_transactions = pd.read_csv("/media/andraa/10160545101605451/kaggle/WSDM-kaggle/data/train_transactions.csv")

In [20]:
train_transactions.head()

,Unnamed: 0,msno,is_churn,payment_method_id,payment_plan_days,plan_list_price,is_auto_renew
0,0,waLDQMmcOu2jLDaV1ddDkgCrB/jl6sD66Xzs0Vqax1Y=,1,38,30,149,0
1,1,QA7uiXy8vIbUSPOkCf9RwQ3FsT8jVq2OxDr8zqa7bRQ=,1,39,30,149,1
2,2,fGwBva6hikQmTJzrbz/2Ezjm5Cth5jZUNvXigKK2AFA=,1,36,30,150,0
3,3,mT5V8rEpa+8wuqi6x0DoVd3H5icMKkE9Prt49UlmK+4=,1,17,410,0,0
4,4,XaPhtGLk/5UvvOYHcONTwsnH97P4eGECeq+BARGItRw=,1,38,30,149,0


In [23]:
columns_to_merge = ["msno","city", "gender", "registered_via", "false_age", "longivity"]
train_transactions_members = train_transactions.merge(members_df[columns_to_merge])

In [24]:
train_transactions_members.to_csv("/media/andraa/10160545101605451/kaggle/WSDM-kaggle/data/train_transactions_members.csv")

In [26]:
train_transactions_members.head()

,Unnamed: 0,msno,is_churn,payment_method_id,payment_plan_days,plan_list_price,is_auto_renew,city,gender,registered_via,false_age,longivity
0,0,waLDQMmcOu2jLDaV1ddDkgCrB/jl6sD66Xzs0Vqax1Y=,1,38,30,149,0,18,female,9,0,4537.0
1,1,QA7uiXy8vIbUSPOkCf9RwQ3FsT8jVq2OxDr8zqa7bRQ=,1,39,30,149,1,10,male,9,0,4366.0
2,2,fGwBva6hikQmTJzrbz/2Ezjm5Cth5jZUNvXigKK2AFA=,1,36,30,150,0,11,female,9,0,4128.0
3,3,mT5V8rEpa+8wuqi6x0DoVd3H5icMKkE9Prt49UlmK+4=,1,17,410,0,0,13,female,9,0,4346.0
4,4,XaPhtGLk/5UvvOYHcONTwsnH97P4eGECeq+BARGItRw=,1,38,30,149,0,3,male,9,0,4291.0


In [27]:
columns = ["payment_method_id",	"payment_plan_days" ,	"plan_list_price" ,	"is_auto_renew", 	"city" ,	"gender", 	"registered_via" ,	"false_age" ,	"longivity"]

In [30]:
model = LogisticRegression()
model = model.fit(train_transactions_members["is_churn"], train_transactions_members[columns])

/home/andraa/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


ValueError: bad input shape (876143, 9)

In [31]:
train_transactions_members["is_churn"]

0         1
1         1
2         1
3         1
4         1
5         1
6         1
7         1
8         1
9         1
10        1
11        1
12        1
13        1
14        1
15        1
16        1
17        1
18        1
19        1
20        1
21        1
22        1
23        1
24        1
25        1
26        1
27        1
28        1
29        1
         ..
876113    0
876114    0
876115    0
876116    0
876117    0
876118    0
876119    0
876120    0
876121    0
876122    0
876123    0
876124    0
876125    0
876126    0
876127    0
876128    0
876129    0
876130    0
876131    0
876132    0
876133    0
876134    0
876135    0
876136    0
876137    0
876138    0
876139    0
876140    0
876141    0
876142    0
Name: is_churn, dtype: int64